In [2]:
import yfinance as yf
from datetime import datetime
import quandl
import stockquotes
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

%matplotlib inline

In [5]:
SP500 = stockquotes.Stock("^GSPC")
S = SP500.current_price
print(f'S&P500 last price was ${S}, thus we will look for strike price in the range of ${round(S-300)} - ${round(S+300)}')

S&P500 last price was $3668.1, thus we will look for strike price in the range of $3368 - $3968


In [51]:
def call_data(ticker,expiration):
    '''
    Extracts the strike price and call premium of an option given the ticker symbol of the underlying asset and 
    the expiration date. Returns a list of strike price and call premium.
    
    Arguments:
    ticker -- String that represents the ticker symbol of the underlying asset. Refer to Yahoo Finance website to find the 
              symbol
    expiration -- String date in format 'YYYY-MM-DD'. Refer to Yahoo Finance to check list of available expirations for 
                  the underlying asset
                  
    Returns:
    data -- list containig pairs of strike price and call premiums available for the underlying in the expiration selected
    '''
    
    security = yf.Ticker(ticker)
    
    if expiration not in security.options:                       #Assertion to verify that expiration exists
        print('Option expiration not available, please verify')
        return
        
    options = security.option_chain(expiration)
    calls = options.calls
    strike_price = calls['strike']
    call_price = calls['lastPrice']
    
    data = list(zip(strike_price, call_price))
    
    return data

In [52]:
call_data('^GSPC','2021-12-16')

[(900.0, 2755.29),
 (1000.0, 2656.54),
 (1225.0, 1956.5),
 (1300.0, 2039.26),
 (1375.0, 2143.78),
 (1400.0, 2119.6),
 (1500.0, 2160.15),
 (1525.0, 1425.66),
 (1550.0, 1201.5),
 (1600.0, 1600.8),
 (1625.0, 1375.0),
 (1650.0, 1054.5),
 (1800.0, 1243.1),
 (1825.0, 1466.0),
 (1900.0, 1361.4),
 (2000.0, 1643.6),
 (2175.0, 779.5),
 (2200.0, 1166.64),
 (2250.0, 1311.3),
 (2300.0, 1321.8),
 (2350.0, 834.0),
 (2400.0, 1007.0),
 (2425.0, 760.89),
 (2475.0, 1074.55),
 (2500.0, 1205.23),
 (2525.0, 575.4),
 (2550.0, 877.68),
 (2575.0, 895.15),
 (2600.0, 1043.62),
 (2625.0, 920.0),
 (2650.0, 909.5),
 (2675.0, 564.9),
 (2700.0, 960.0),
 (2725.0, 756.5),
 (2750.0, 966.06),
 (2775.0, 371.06),
 (2800.0, 945.42),
 (2825.0, 814.86),
 (2850.0, 851.1),
 (2900.0, 813.82),
 (2925.0, 629.0),
 (2950.0, 797.0),
 (3000.0, 758.0),
 (3100.0, 670.9),
 (3200.0, 622.93),
 (3300.0, 550.0),
 (3400.0, 480.0),
 (3500.0, 406.26),
 (3600.0, 329.1),
 (3900.0, 159.0),
 (4000.0, 118.0)]

In [49]:
def Risk_free(ttm):
    '''
    Extracts the most recent US treasury rate given the time to maturity.
    
    Arguments:
    ttm -- Time to maturity in string format. The available options are:
    ['1 MO', '2 MO', '3 MO', '6 MO', '1 YR', '2 YR', '3 YR', '5 YR', '7 YR', '10 YR', '20 YR', '30 YR']
    
    Returns:
    rf -- Most recent risk free rate for the time to maturity selected
    '''
    yield_curve = quandl.get('USTREASURY/YIELD', authtoken='JMxryiBcRV26o9r5q7uv')
    rf = yield_curve[ttm][-1]
    
    return rf    

In [25]:
Risk_free('1 YR')

0.1

In [32]:
def Time_to_maturity(expiration):
    '''
    Calculates the time to maturity in years from today to the option expiration
    
    Arguments:
    expiration -- String date in format 'YYYY-MM-DD'. Refer to Yahoo Finance to check list of available expirations for 
                  the underlying asset
                  
    Returns:
    ttm -- Time to maturity in year in float format
    '''
    
    today = datetime.now()
    expiration_d = datetime.strptime(expiration,'%Y-%m-%d')
    datetime_delta = (expiration_d - today).days
    ttm = datetime_delta/365
    
    return ttm
    

In [33]:
Time_to_maturity('2021-12-16')

1.010958904109589